## Linear Regression Consulting Project

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
spark = SparkSession.builder.appName('logreg').getOrCreate()
df = spark.read.csv('/FileStore/tables/customer_churn.csv', inferSchema=True, header=True)
df.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [3]:
df.count()

Out[55]: 900

In [4]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+

In [5]:
# Change Onboard_date to year of onboard
from pyspark.sql.functions import year
df = df.withColumn('Onboard_year', year(df['Onboard_date']))

In [6]:
df.columns

Out[20]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'Onboard_year']

In [7]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites','Onboard_year'], outputCol='features')
output = assembler.transform(df)

In [8]:
final_data = output.select('features', 'Churn')

In [9]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [10]:
model = LogisticRegression(labelCol='Churn')
fit_model = model.fit(train_data)

In [11]:
train_summary = fit_model.summary
train_summary.predictions.describe().show()

+-------+-------------------+-------------------+
summary| Churn| prediction|
+-------+-------------------+-------------------+
 count| 632| 632|
 mean|0.14556962025316456|0.10126582278481013|
 stddev|0.35295356661317195|0.30191934183271096|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pred = fit_model.evaluate(test_data)
pred.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[26.0,8787.39,1.0...| 1|[1.23248281130936...|[0.77425282887087...| 0.0|
[26.0,8939.61,0.0...| 0|[6.34530115268474...|[0.99824810039057...| 0.0|
[28.0,11128.95,1....| 0|[4.46375377290949...|[0.98861213513441...| 0.0|
[28.0,11204.23,0....| 0|[2.02546628382362...|[0.88344505261879...| 0.0|
[29.0,5900.78,1.0...| 0|[4.38181771764628...|[0.98765177334780...| 0.0|
[29.0,9378.24,0.0...| 0|[4.85220454034462...|[0.99224940240783...| 0.0|
[29.0,12711.15,0....| 0|[5.43433721869296...|[0.99565483631975...| 0.0|
[30.0,8677.28,1.0...| 0|[4.49940857813235...|[0.98900662898454...| 0.0|
[30.0,10744.14,1....| 1|[2.24229224271517...|[0.90398360297639...| 0.0|
[30.0,10960.52,1....| 0|[2.78053346824197...|[0.94161477952414...| 0.0|
[30.0,11575.37,1....| 1|[4.30280785197452...|[0.98665011683586...| 0.0|
[30.0,12788.37,0....| 0|[2.61179875217844...|[0.93161707784234...| 0.0|
[31.0,7073.61,0.0...| 0|[3.25055535175160...|[0.96269306331171...| 0.0|
[31.0,8829.83,1.0...| 0|[4.57834131419495...|[0.98983251948512...| 0.0|
[31.0,10058.87,1....| 0|[4.76337037094634...|[0.99153547118213...| 0.0|
[31.0,10182.6,1.0...| 0|[4.92036791135778...|[0.99275640657997...| 0.0|
[31.0,11743.24,0....| 0|[6.64329692895284...|[0.99869896972128...| 0.0|
[32.0,8011.38,0.0...| 0|[2.20641919641783...|[0.90082447760582...| 0.0|
[32.0,8617.98,1.0...| 1|[1.49049219887257...|[0.81615213744388...| 0.0|
[32.0,9885.12,1.0...| 1|[2.21671162178348...|[0.90174021427762...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [13]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='Churn')
auc = churn_eval.evaluate(pred.predictions)
auc

Out[39]: 0.7529556650246305

In [14]:
# Fit model on the entire dataset
final_lr_model = model.fit(final_data)

In [15]:
new_customers = spark.read.csv('/FileStore/tables/new_customers.csv',inferSchema=True,header=True)
new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [16]:
new_customers = new_customers.withColumn('Onboard_year', year(new_customers['Onboard_date']))

In [17]:
new_customers = assembler.transform(new_customers)
new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Onboard_year: integer (nullable = true)
-- features: vector (nullable = true)

In [18]:
final_results = final_lr_model.transform(new_customers)
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+------------+--------------------+--------------------+--------------------+----------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Onboard_year| features| rawPrediction| probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+------------+--------------------+--------------------+--------------------+----------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd| 2011|[37.0,9935.53,1.0...|[1.60052476885617...|[0.83209171604081...| 0.0|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson| 2013|[23.0,7526.94,1.0...|[1.52117433817519...|[0.82071134276875...| 0.0|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson| 2006|[65.0,100.0,1.0,1...|[1.55336434080440...|[0.82539911612627...| 0.0|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden| 2016|[32.0,6487.5,0.0,...|[1.53271055647258...|[0.82240255488448...| 0.0|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC| 2012|[32.0,13147.71,1....|[1.58762913431331...|[0.83028227736626...| 0.0|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins| 2011|[22.0,8445.26,1.0...|[1.56891586790924...|[0.82762900173932...| 0.0|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+------------+--------------------+--------------------+--------------------+----------+

We should assign Acocunt Managers to companys who are predicted to churn.